# SHUBHAM SHARMA
## IIT BOMBAY
This code is a pytorch implimentation of the U-Net architecture that we have

In [1]:
#Importing the functions for U-net
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
# from torchsummary import summary
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#CHecking the number of GPU and then setting the GPU id
print(torch.cuda.current_device())#To know thw current active device
device = torch.device('cuda:1')
print(device)

0
cuda:1


In [4]:
num_of_classes=2
# input_size=(1, 128, 128, 128)
# input_size[1]*input_size[2]*input_size[3]

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        
        #For first layer input
        self.conv1a=nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv1b=nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN1=nn.BatchNorm3d(32)
        
        #For second layer 
        self.conv2a=nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv2b=nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN2=nn.BatchNorm3d(64)
        
        #For third layer 
        self.conv3a=nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv3b=nn.Conv3d(in_channels=128, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN3=nn.BatchNorm3d(128)
        
        #For forth layer 
        self.conv4a=nn.Conv3d(in_channels=128, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv4b=nn.Conv3d(in_channels=256, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN4=nn.BatchNorm3d(256)
        
        #For fifth/latent layer 
        self.conv5a=nn.Conv3d(in_channels=256, out_channels=512, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv5b=nn.Conv3d(in_channels=512, out_channels=512, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For Upsampling/ sixth layer
        self.upconv1=nn.ConvTranspose3d(in_channels=512, out_channels=256, kernel_size=(2,2,2), stride=2, padding=0)
        self.conv6a=nn.Conv3d(in_channels=512, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv6b=nn.Conv3d(in_channels=256, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For Upsampling/ seventh layer
        self.upconv2=nn.ConvTranspose3d(in_channels=256, out_channels=128, kernel_size=(2,2,2), stride=2, padding=0)
        self.conv7a=nn.Conv3d(in_channels=256, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv7b=nn.Conv3d(in_channels=128, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For Upsampling/ eighth layer
        self.upconv3=nn.ConvTranspose3d(in_channels=128, out_channels=64, kernel_size=(2,2,2), stride=2, padding=0)
        self.conv8a=nn.Conv3d(in_channels=128, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv8b=nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)

        #For Upsampling/ ninth layer
        self.upconv4=nn.ConvTranspose3d(in_channels=64, out_channels=32, kernel_size=(2,2,2), stride=2, padding=0)
        self.conv9a=nn.Conv3d(in_channels=64, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv9b=nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For last layer
        self.convLast=nn.Conv3d(in_channels=32, out_channels=num_of_classes, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For maxpool
        self.max_pool=nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        
        #For activation function
        self.relu = nn.ReLU()
        
    def forward(self,x):
        conv1 = self.conv1a(x)
        conv1 = self.relu(conv1)
        conv1 = self.BN1(conv1)
        conv1 = self.conv1b(conv1)
        conv1 = self.relu(conv1)
        conv1 = self.BN1(conv1)
        pool1 = self.max_pool(conv1)
        ##
        conv2 = self.conv2a(pool1)
        conv2 = self.relu(conv2)
        conv2 = self.BN2(conv2)
        conv2 = self.conv2b(conv2)
        conv2 = self.relu(conv2)
        conv2 = self.BN2(conv2)
        pool2 = self.max_pool(conv2)
        ##
        conv3 = self.conv3a(pool2)
        conv3 = self.relu(conv3)
        conv3 = self.BN3(conv3)
        conv3 = self.conv3b(conv3)
        conv3 = self.relu(conv3)
        conv3 = self.BN3(conv3)
        pool3 = self.max_pool(conv3)
        #
        conv4 = self.conv4a(pool3)
        conv4 = self.relu(conv4)
        conv4 = self.BN4(conv4)
        conv4 = self.conv4b(conv4)
        conv4 = self.relu(conv4)
        conv4 = self.BN4(conv4)
        pool4 = self.max_pool(conv4)
        ##
        conv5 = self.conv5a(pool4)
        conv5 = self.relu(conv5)
        conv5 = self.conv5b(conv5)
        conv5 = self.relu(conv5)
        ##
        up6 = torch.cat((self.upconv1(conv5),conv4),dim=1)
        conv6 = self.conv6a(up6)
        conv6 = self.conv6b(conv6)
        ##
        up7 = torch.cat((self.upconv2(conv6),conv3),dim=1)
        conv7 = self.conv7a(up7)
        conv7 = self.conv7b(conv7)
        ##
        up8 = torch.cat((self.upconv3(conv7),conv2),dim=1)
        conv8 = self.conv8a(up8)
        conv8 = self.conv8b(conv8)
        ##
        up9 = torch.cat((self.upconv4(conv8),conv1),dim=1)
        conv9 = self.conv9a(up9)
        conv9 = self.conv9b(conv9)
        ##
        conv10 = self.convLast(conv9)
#         conv10 = conv10.view(-1,input_size[1]*input_size[2]*input_size[3])
        return conv10
          
          
          
##########################################################################
model = Net().to(device)#not necessary to add to device
# summary(model, input_size=input_size)        
    
        
        

In [3]:
num_of_classes=2
# input_size=(1, 128, 128, 128)
# input_size[1]*input_size[2]*input_size[3]

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        
        #For first layer input(_,320,320,128)
        self.conv1a=nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv1b=nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN1=nn.BatchNorm3d(32)
        
        #For second layer (_,160,160,64)
        self.conv2a=nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv2b=nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN2=nn.BatchNorm3d(64)
        
        #For third layer (80,80,32)
        self.conv3a=nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv3b=nn.Conv3d(in_channels=128, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN3=nn.BatchNorm3d(128)
        
        #For forth layer 
        self.conv4a=nn.Conv3d(in_channels=128, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv4b=nn.Conv3d(in_channels=256, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1)
        self.BN4=nn.BatchNorm3d(256)
        
        #For Upsampling/ seventh layer
        self.upconv1=nn.ConvTranspose3d(in_channels=256, out_channels=128, kernel_size=(2,2,2), stride=2, padding=0)
        self.conv5a=nn.Conv3d(in_channels=256, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv5b=nn.Conv3d(in_channels=128, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For Upsampling/ eighth layer
        self.upconv2=nn.ConvTranspose3d(in_channels=128, out_channels=64, kernel_size=(2,2,2), stride=2, padding=0)
        self.conv6a=nn.Conv3d(in_channels=128, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv6b=nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1)

        #For Upsampling/ ninth layer
        self.upconv3=nn.ConvTranspose3d(in_channels=64, out_channels=32, kernel_size=(2,2,2), stride=2, padding=0)
        self.conv7a=nn.Conv3d(in_channels=64, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        self.conv7b=nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For last layer
        self.convLast=nn.Conv3d(in_channels=32, out_channels=num_of_classes, kernel_size=(3,3,3), stride=1, padding=1)
        
        #For maxpool
        self.max_pool=nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        
        #For activation function
        self.relu = nn.ReLU()
        
    def forward(self,x):
        conv1 = self.conv1a(x)
        conv1 = self.relu(conv1)
        conv1 = self.BN1(conv1)
        conv1 = self.conv1b(conv1)
        conv1 = self.relu(conv1)
        conv1 = self.BN1(conv1)
        pool1 = self.max_pool(conv1)
        ##
        conv2 = self.conv2a(pool1)
        conv2 = self.relu(conv2)
        conv2 = self.BN2(conv2)
        conv2 = self.conv2b(conv2)
        conv2 = self.relu(conv2)
        conv2 = self.BN2(conv2)
        pool2 = self.max_pool(conv2)
        ##
        conv3 = self.conv3a(pool2)
        conv3 = self.relu(conv3)
        conv3 = self.BN3(conv3)
        conv3 = self.conv3b(conv3)
        conv3 = self.relu(conv3)
        conv3 = self.BN3(conv3)
        pool3 = self.max_pool(conv3)
        #
        conv4 = self.conv4a(pool3)
        conv4 = self.relu(conv4)
        conv4 = self.conv4b(conv4)
        conv4 = self.relu(conv4)
        ##
        up5 = torch.cat((self.upconv1(conv4),conv3),dim=1)
        conv5 = self.conv5a(up5)
        conv5 = self.conv5b(conv5)
        ##
        up6 = torch.cat((self.upconv2(conv5),conv2),dim=1)
        conv6 = self.conv6a(up6)
        conv6 = self.conv6b(conv6)
        ##
        up6 = torch.cat((self.upconv3(conv6),conv1),dim=1)
        conv7 = self.conv7a(up6)
        conv7 = self.conv7b(conv7)
        ##
        conv8 = self.convLast(conv7)
        return conv7
          
          
          
##########################################################################
model = Net().to(device)#not necessary to add to device
# summary(model, input_size=input_size)        
    
        
        

In [4]:
#Loading the dataset
X=np.load('./X_cardiac.npy')
X=X.reshape(X.shape[0],1,X.shape[1],X.shape[2],X.shape[3])
X_whitened = np.zeros(X.shape)
for i in range(X.shape[0]):
    temp1=X[i]
    temp2=(temp1 - np.mean(temp1))/(np.std(temp1))
    X_whitened[i]=temp2

Y=np.load('./Y_cardiac.npy')

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X_whitened, Y, test_size=0.20)
print('The shape of x_train_cardiac is :',x_train.shape)
print('The shape of x_test_cardiac is :',x_test.shape)
print('The shape of y_train_cardiac is :',y_train.shape)
print('The shape of y_test_cardiac is :',y_test.shape)
x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

The shape of x_train_cardiac is : (16, 1, 320, 320, 128)
The shape of x_test_cardiac is : (4, 1, 320, 320, 128)
The shape of y_train_cardiac is : (16, 320, 320, 128)
The shape of y_test_cardiac is : (4, 320, 320, 128)


In [6]:
trainset = torch.utils.data.TensorDataset(x_train,y_train)# create your datset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

valset = torch.utils.data.TensorDataset(x_test,y_test)# create your datset
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=True)

In [7]:
# learning rate and number of  epochs
num_of_epochs = 1
lr=1e-4

# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()#NO NEED TO BE USING SOFTMAX WHEN USING Crioss Entropy loss
optimizer = optim.Adam(model.parameters(), lr=lr) 

In [8]:
#Training the model
total_step = len(trainloader)
for epochs in range(num_of_epochs):
    for i, (images, labels) in enumerate(trainloader):  
            #CHECK THE SHAPE OF BOTH IMAGES AND LABELS
            images = images.to(device)
            images= images.float()
            labels = labels.to(device)  

            #Forward pass
            outputs = model(images)
            labels = labels.long()
            loss = loss_fn(outputs, labels)

            # Backpropagation and then optimization
            optimizer.zero_grad()#Initially setting the gradient values to zero so backward() can find the gradient
            loss.backward()#backpropagate and then optimize
            optimizer.step()
        #     if (i+1) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epochs+1, num_of_epochs, i+1, total_step, loss.item()))
    torch.save(model, './cardiac')

    
    
    

RuntimeError: CUDA error: out of memory

In [10]:
a = x_train[1]
a.shape

torch.Size([1, 320, 320, 128])